<a href="https://colab.research.google.com/github/Malanidhruv/quant-projects/blob/main/NIFTY_500_Stocks_having_CAGR_of_20_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pya3 nsetools quantstats seaborn matplotlib holidays


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00


In [2]:
pip install quantstats==0.0.50

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: quantstats
    Found existing installation: QuantStats 0.0.62
    Uninstalling QuantStats-0.0.62:
      Successfully uninstalled QuantStats-0.0.62


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pya3 import Aliceblue
from nsetools import Nse
from datetime import datetime, timedelta
import holidays
import quantstats as qs


In [4]:
def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key


In [8]:

import re
import io
import contextlib



def parse_metrics(report):
    metrics = {}
    lines = report.split('\n')
    for line in lines:
        match = re.match(r'(.+?)\s+([\d\.\-\%]+)', line)
        if match:
            key, value = match.groups()
            metrics[key.strip()] = value.strip()
    return metrics

def evaluate_stock(metrics):
    criteria = {
        'CAGR﹪': 20,           # Compound Annual Growth Rate
        'Sharpe': 1,           # Sharpe Ratio
        'Sortino': 1           # Sortino Ratio
    }

    # Extract relevant metrics
    cagr = float(metrics['CAGR﹪'].strip('%'))
    sharpe = float(metrics['Sharpe'])
    sortino = float(metrics['Sortino'])

    # Evaluate criteria
    if (cagr > criteria['CAGR﹪'] and
        sharpe > criteria['Sharpe'] and
        sortino > criteria['Sortino']):
        return "Buy"
    else:
        return "Do Not Buy"

def main():
    # Get user credentials
    user_id, api_key = get_user_credentials()

    # Create an instance of Aliceblue
    alice = Aliceblue(user_id=user_id, api_key=api_key)

    # Get session ID
    alice.get_session_id()

    # Create an instance of Nse
    nse = Nse()

    # Define instrument and time range for historical data
    instrument = alice.get_instrument_by_token('NSE', 1624)
    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 22)
    interval = "D"

    # Fetch historical data
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Create DataFrame
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Capture the metrics report as a string
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        qs.reports.metrics(mode='basic|full', returns=df['pct'])
    report = buffer.getvalue()

    # Print the captured report in the notebook for debugging
    print("Captured Report:\n", report)

    # Parse the metrics from the report
    metrics = parse_metrics(report)

    # Print the parsed metrics for debugging
    print("Parsed Metrics:", metrics)

    # Make a buy recommendation based on metrics
    recommendation = evaluate_stock(metrics)
    print("Recommendation:", recommendation)

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: yF7VbKapUqpMvaGv53IMNUK1bSQQiq6VxpW3aCnYHdPCAHWgUEmTA8l8ESoeNCpXLRRqeHhIlycWKAURazd9Pg7WuPTA1V1ks3GATkrNsdGSpzEm59rgkssHOXc5IxLa
Captured Report:
                     Strategy
------------------  ----------
Start Period        2014-07-02
End Period          2024-07-22
Risk-Free Rate      0.0%
Time in Market      100.0%

Cumulative Return   -50.92%
CAGR﹪              -6.83%

Sharpe              0.06
Sortino             0.07
Sortino/√2          0.05
Omega               1.01

Max Drawdown        -89.89%
Longest DD Days     2845

Gain/Pain Ratio     0.01
Gain/Pain (1M)      0.05

Payoff Ratio        0.97
Profit Factor       1.01
Common Sense Ratio  1.11
CPC Index           0.5
Tail Ratio          1.1
Outlier Win Ratio   3.63
Outlier Loss Ratio  3.43

MTD                 1.55%
3M                  0.92%
6M                  13.14%
YTD                 29.53%
1Y                  69.29%
3Y (ann.)           16.38%
5Y (ann.)           2.68%
10Y (ann.

In [15]:
import re
import io
import contextlib
import pandas as pd


def parse_metrics(report):
    metrics = {}
    lines = report.split('\n')
    for line in lines:
        match = re.match(r'(.+?)\s+([\d\.\-\%]+)', line)
        if match:
            key, value = match.groups()
            metrics[key.strip()] = value.strip()
    return metrics

def evaluate_stock(metrics, stock_name):
    criteria = {
        'CAGR﹪': 20,           # Compound Annual Growth Rate
        'Sharpe': 1,           # Sharpe Ratio
        'Sortino': 1           # Sortino Ratio
    }

    # Extract relevant metrics
    cagr = float(metrics['CAGR﹪'].strip('%'))
    sharpe = float(metrics['Sharpe'])
    sortino = float(metrics['Sortino'])

    # Evaluate criteria
    if (cagr > criteria['CAGR﹪'] and
        sharpe > criteria['Sharpe'] and
        sortino > criteria['Sortino']):
        return f"Buy {stock_name}"
    else:
        return f"Do Not Buy {stock_name}"

def main():
    # Get user credentials
    user_id, api_key = get_user_credentials()

    # Create an instance of Aliceblue
    alice = Aliceblue(user_id=user_id, api_key=api_key)

    # Get session ID
    alice.get_session_id()

    # Create an instance of Nse
    nse = Nse()

    # Define instrument and time range for historical data
    instrument_token = 1624  # Example token
    instrument = alice.get_instrument_by_token('NSE', instrument_token)
    stock_name = instrument.symbol  # Extract stock name

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 22)
    interval = "D"

    # Fetch historical data
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Create DataFrame
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Capture the metrics report as a string
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        qs.reports.metrics(mode='basic|full', returns=df['pct'])
    report = buffer.getvalue()

    # Print the captured report in the notebook for debugging
    print("Captured Report:\n", report)

    # Parse the metrics from the report
    metrics = parse_metrics(report)

    # Print the parsed metrics for debugging
    print("Parsed Metrics:", metrics)

    # Make a buy recommendation based on metrics
    recommendation = evaluate_stock(metrics, stock_name)
    print("Recommendation:", recommendation)

if __name__ == "__main__":
    main()


Enter your user ID: 1141826
Enter your API key: yF7VbKapUqpMvaGv53IMNUK1bSQQiq6VxpW3aCnYHdPCAHWgUEmTA8l8ESoeNCpXLRRqeHhIlycWKAURazd9Pg7WuPTA1V1ks3GATkrNsdGSpzEm59rgkssHOXc5IxLa
Captured Report:
                     Strategy
------------------  ----------
Start Period        2014-07-02
End Period          2024-07-22
Risk-Free Rate      0.0%
Time in Market      100.0%

Cumulative Return   -50.92%
CAGR﹪              -6.83%

Sharpe              0.06
Sortino             0.07
Sortino/√2          0.05
Omega               1.01

Max Drawdown        -89.89%
Longest DD Days     2845

Gain/Pain Ratio     0.01
Gain/Pain (1M)      0.05

Payoff Ratio        0.97
Profit Factor       1.01
Common Sense Ratio  1.11
CPC Index           0.5
Tail Ratio          1.1
Outlier Win Ratio   3.63
Outlier Loss Ratio  3.43

MTD                 1.55%
3M                  0.92%
6M                  13.14%
YTD                 29.53%
1Y                  69.29%
3Y (ann.)           16.38%
5Y (ann.)           2.68%
10Y (ann.

In [17]:
import re
import io
import contextlib
import pandas as pd
from datetime import datetime


def get_user_credentials():
    # Replace with actual method to retrieve user credentials
    user_id = '1141826'
    api_key = 'yF7VbKapUqpMvaGv53IMNUK1bSQQiq6VxpW3aCnYHdPCAHWgUEmTA8l8ESoeNCpXLRRqeHhIlycWKAURazd9Pg7WuPTA1V1ks3GATkrNsdGSpzEm59rgkssHOXc5IxLa'
    return user_id, api_key

def parse_metrics(report):
    metrics = {}
    lines = report.split('\n')
    for line in lines:
        match = re.match(r'(.+?)\s+([\d\.\-\%]+)', line)
        if match:
            key, value = match.groups()
            metrics[key.strip()] = value.strip()
    return metrics

def evaluate_stock(metrics, stock_name):
    criteria = {
        'CAGR﹪': 20,           # Compound Annual Growth Rate
        'Sharpe': 1,            # Sharpe Ratio
        'Sortino': 1            # Sortino Ratio
    }

    # Extract relevant metrics
    cagr = float(metrics['CAGR﹪'].strip('%'))
    sharpe = float(metrics['Sharpe'])
    sortino = float(metrics['Sortino'])

    # Evaluate criteria
    if (cagr > criteria['CAGR﹪'] and
        sharpe > criteria['Sharpe'] and
        sortino > criteria['Sortino']):
        return f"Buy {stock_name}"
    else:
        return f"Do Not Buy {stock_name}"

def analyze_stock(alice, stock_token):
    # Define instrument and time range for historical data
    instrument = alice.get_instrument_by_token('NSE', stock_token)
    stock_name = instrument.symbol  # Extract stock name

    from_datetime = datetime(2014, 7, 1)
    to_datetime = datetime(2024, 7, 22)
    interval = "D"

    # Fetch historical data
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)

    # Create DataFrame
    df = pd.DataFrame(historical_data)

    # Preprocess data
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df['pct'] = df['close'].pct_change()
    df.columns = df.columns.str.strip()
    df = df.drop(columns=['open', 'high', 'low', 'close', 'volume']).dropna()

    # Capture the metrics report as a string
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        qs.reports.metrics(mode='basic|full', returns=df['pct'])
    report = buffer.getvalue()

    # Parse the metrics from the report
    metrics = parse_metrics(report)

    # Make a buy recommendation based on metrics
    recommendation = evaluate_stock(metrics, stock_name)
    return recommendation

def main():
    # Load CSV file
    file_path = '/content/NIFTY 500 stocks.csv'
    stocks_df = pd.read_csv(file_path)

    # Get user credentials
    user_id, api_key = get_user_credentials()

    # Create an instance of Aliceblue
    alice = Aliceblue(user_id=user_id, api_key=api_key)

    # Get session ID
    alice.get_session_id()

    # Iterate over each stock token
    recommendations = []
    for _, row in stocks_df.iterrows():
        stock_token = int(row['Symbol'])  # Adjust column name if necessary
        try:
            recommendation = analyze_stock(alice, stock_token)
            recommendations.append((stock_token, recommendation))
        except Exception as e:
            print(f"Error analyzing stock {stock_token}: {e}")

    # Print recommendations
    for stock_token, recommendation in recommendations:
        print(f"Stock {stock_token}: {recommendation}")

if __name__ == "__main__":
    main()


Stock 5610: Do Not Buy ASTRAZEN
Stock 14111: Do Not Buy NETWORK18
Stock 7227: Do Not Buy MAPMYINDIA
Stock 5378: Do Not Buy ASAHIINDIA
Stock 3149: Do Not Buy BORORENEW
Stock 7401: Do Not Buy RAJESHEXPO
Stock 2319: Do Not Buy NCC
Stock 3506: Do Not Buy TITAN
Stock 1660: Do Not Buy ITC
Stock 3339: Do Not Buy SUNDARMFIN
Stock 2792: Do Not Buy EASEMYTRIP
Stock 1808: Do Not Buy KAJARIACER
Stock 12018: Do Not Buy SUZLON
Stock 7936: Do Not Buy AVANTIFEED
Stock 324: Do Not Buy ANGELONE
Stock 17704: Do Not Buy QUESS
Stock 14208: Do Not Buy TV18BRDCST
Stock 3432: Buy TATACONSUM
Stock 3518: Do Not Buy TORNTPHARM
Stock 2955: Buy KALYANKJIL
Stock 2783: Do Not Buy JUBLINGREA
Stock 6596: Do Not Buy SONATSOFTW
Stock 18060: Buy CONCORDBIO
Stock 29135: Do Not Buy INDUSTOWER
Stock 19020: Buy JSWINFRA
Stock 11571: Do Not Buy PPLPHARMA
Stock 18721: Buy NUVAMA
Stock 14154: Buy UNOMINDA
Stock 10099: Do Not Buy GODREJCP
Stock 4421: Do Not Buy CREDITACC
Stock 12092: Buy KAYNES
Stock 1153: Do Not Buy GLAXO
Stock